In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, RocCurveDisplay

In [5]:
df = pd.read_csv("../Oblig1/wdbc.data", header=None)
df["target"] = np.where(df[1] == "M", 1, 0)
df = df.drop([0,1], axis=1)

Først leser vi inn datasetet og ser hvilke verdier vi h. I og konvertere verdiene i target kollonen til 1 for M og 0 for B, slik at disse er på et format som kan brukes senere, vi legger også disse verdiene i en ny kollone slik at vi kan bruke disse  Vi tar også og droppe kollonen for ID-veriden og den gamle kollonen med target verdiene



ref for np.where: https://www.geeksforgeeks.org/numpy/numpy-where-in-python/

In [111]:
def split_dataset(data):
    train_val = data.groupby("target", group_keys=False)[data.columns].sample(frac=0.8)
    test = data.drop(train_val.index)
    train = train_val.groupby("target", group_keys=False)[train_val.columns].sample(frac=0.75)
    validation = train_val.drop(train.index)    
    return [train, validation, test]

def check_split_dataset(data):
    print(data[0]["target"].value_counts()/data[0].shape[0])
    print(data[1]["target"].value_counts()/data[1].shape[0])
    print(data[2]["target"].value_counts()/data[2].shape[0])

def split_targets_and_features(data):
    targets = data.target
    features = data.drop("target", axis=1)
    return [targets, features]

def create_set_of_targets_and_features(data):
    train = split_targets_and_features(data[0])
    validation = split_targets_and_features(data[1])
    test = split_targets_and_features(data[2])

    return [train, validation, test]

def create_full_dataset(input):
    data = split_dataset(input)
    dataset = create_set_of_targets_and_features(data)

    return dataset
        
def evaluate_model(classifier, features, targets):
    predictions = classifier.predict(features)
    accuracy = accuracy_score(targets, predictions)
    precision = precision_score(targets, predictions)
    recall = recall_score(targets, predictions)
    f1 = f1_score(targets, predictions)
    roc_auc = roc_auc_score(targets, predictions)   

    return [accuracy, precision, recall, f1, roc_auc]

def multiple_evaluate_model(classifier, features, targets):
    accuracy, precision, recall, f1, roc_auc = [], [], [], [], []

    for i in range(0,30,1):
        result = evaluate_model(classifier, features, targets)
        accuracy.append(result[0])
        precision.append(result[1])
        recall.append(result[2])
        f1.append(result[3])
        roc_auc.append(result[4])

    print(len(accuracy))
    print(f'Accuracy: {np.mean(accuracy):.2f}')
    print(f'Precision: {np.mean(precision):.2f}')
    print(f'Recall: {np.mean(recall):.2f}')
    print(f'F1: {np.mean(f1):.2f}')
    print(f'ROCAUC: {np.mean(roc_auc):.2f}')

def append_values(results, accuracy, precision, recall, f1, roc_auc):
        accuracy.append(results[0])
        precision.append(results[1])
        recall.append(results[2])
        f1.append(results[3])
        roc_auc.append(results[4])

def results_print(accuracy, precision, recall, f1, roc_auc):
    print(f'Accuracy: {np.mean(accuracy):.2f}')
    print(f'Precision: {np.mean(precision):.2f}')
    print(f'Recall: {np.mean(recall):.2f}')
    print(f'F1: {np.mean(f1):.2f}')
    print(f'ROCAUC: {np.mean(roc_auc):.2f}')

def test_results_print(accuracy, precision, recall, f1, roc_auc):
    print(f'Accuracy: {np.mean(accuracy):.2f} +- {np.std(accuracy):.2f}')
    print(f'Precision: {np.mean(precision):.2f} +- {np.std(precision):.2f}')
    print(f'Recall: {np.mean(recall):.2f} +- {np.std(recall):.2f}')
    print(f'F1: {np.mean(f1):.2f} +- {np.std(f1):.2f}')
    print(f'ROCAUC: {np.mean(roc_auc):.2f} +- {np.std(roc_auc):.2f}')

def validate(classifier):    
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = [], [], [], [], []
    validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc = [], [], [], [], []

    for i in range(0, 10, 1):
        dataset = create_full_dataset(df)
        classifier.fit(dataset[0][1], dataset[0][0])
    
        results_train = evaluate_model(classifier, dataset[0][1], dataset[0][0])
        append_values(results_train, train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
    
        results_validate = evaluate_model(classifier, dataset[1][1], dataset[1][0])
        append_values(results_validate, validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc)

    print('Training results:')
    results_print(train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
    print('Validation results:')
    results_print(validate_accuracy, validate_precision, validate_recall, validate_f1, validate_roc_auc)

def test(classifier):
    accuracy, precision, recall, f1, roc_auc = [], [], [], [], []

    for i in range(0, 10, 1):
        dataset = create_full_dataset(df)
        classifier.fit(dataset[0][1], dataset[0][0])

        results = evaluate_model(classifier, dataset[2][1], dataset[2][0])
        append_values(results, accuracy, precision, recall, f1, roc_auc)
    
    print('Test results:')
    test_results_print(accuracy, precision, recall, f1, roc_auc)


In [112]:

dt1 = DecisionTreeClassifier()
validate(dt1)
test(dt1)



Training results:
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1: 1.00
ROCAUC: 1.00
Validation results:
Accuracy: 0.93
Precision: 0.91
Recall: 0.90
F1: 0.90
ROCAUC: 0.92
Test results:
Accuracy: 0.92 +- 0.03
Precision: 0.90 +- 0.04
Recall: 0.90 +- 0.04
F1: 0.90 +- 0.04
ROCAUC: 0.92 +- 0.03


In [2]:
df2 = pd.DataFrame({"test": [1,2,3], "test2": [4,5,6]})
df2.head()


,test,test2
0,1,4
1,2,5
2,3,6


In [6]:


print(np.mean(df[3]))

19.289648506151142
